In [51]:
import pandas as pd
import spacy
from nltk import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from html import unescape
from spacy.lang.en import STOP_WORDS

In [2]:
df = pd.read_csv('data/Sentiment-Analysis-Dataset.csv', error_bad_lines=False, engine='python')

Skipping line 4286: ',' expected after '"'
Skipping line 4288: ',' expected after '"'
Skipping line 4290: ',' expected after '"'
Skipping line 4291: ',' expected after '"'
Skipping line 4292: ',' expected after '"'
Skipping line 5182: ',' expected after '"'
Skipping line 8835: ',' expected after '"'
Skipping line 8836: ',' expected after '"'
Skipping line 535876: ',' expected after '"'
Skipping line 535878: ',' expected after '"'
Skipping line 535879: ',' expected after '"'
Skipping line 535880: ',' expected after '"'
Skipping line 535882: ',' expected after '"'


In [7]:
df.columns = ['ItemID', 'Sentiment', 'SentimentSource', 'SentimentText']

In [10]:
df['SentimentText'].head()

0                         is so sad for my APL frie...
1                       I missed the New Moon trail...
2                              omg its already 7:30 :O
3              .. Omgaga. Im sooo  im gunna CRy. I'...
4             i think mi bf is cheating on me!!!   ...
Name: SentimentText, dtype: object

In [13]:
tweet_tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True)

In [18]:
#Parse html from tweets
def processor(doc):
    return unescape(doc).lower()

In [41]:
nlp = spacy.load('en', disable=['ner', 'parser', 'tagger'])

In [42]:
def lemmatizer(doc):
    return [word.lemma_ for word in nlp(doc)]

In [50]:
STOP_WORDS_lemma = [word.lemma_ for word in nlp(' '.join(list(STOP_WORDS)))]
STOP_WORDS_lemma = set(STOP_WORDS_lemma).union({',', '.', ';'})

In [54]:
vectorizer = TfidfVectorizer(preprocessor=processor,
                             tokenizer=lemmatizer,
                             ngram_range=(1, 2),
                             stop_words=STOP_WORDS_lemma)
clf = MultinomialNB()
pipe = Pipeline([('vectorizer', vectorizer),
                 ('classifier', clf)])

In [56]:
X = df['SentimentText']
y = df['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [57]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2',
        preprocessor=<function...True, vocabulary=None)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [58]:
pipe.score(X_train, y_train)

0.908765572870476

In [61]:
pipe.score(X_test, y_test)

0.7699587737013209

In [63]:
import gzip
import dill

In [65]:
with gzip.open('sentiment_model.dill.gz', 'wb') as f:
    dill.dump(pipe, f, recurse=True)